# Assignment3: A\*, IDS, and Effective Branching Factor

Yashad Samant

## OVERVIEW

For this assignment, implement the Recursive Best-First Search using A* algorithm given in class while also implementing the iterative deep learning algorithm done in assignment 2. We have also implemented effective branching factor based on the number of nodes and depth of the problem. Then, we print the data obtained in table form.


## REQUIRED CODE

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. For this you must include your implementations of these functions, from Assignment 2:

Here, we have defined six functions.
* printState_8p - This function converts the original list in an array form so that we can get the exact position of the blank space.

* findBlank_8p - This function iterates through the array and find the blank space.

* actionState_8p - It is responsible for finding the valid actions based on the blank space. It's just made out of different checks.

* takeAction_8p - Based on the obtained valid action, it just swaps the position with the corresponding element and returns the list.

* goalTestF_8p - Returns true if goal == state

* findIndex - Used in heuristic functions to get the two dimensional index of the list.


 - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3]) is used to format the data.
   


## DESIRED OUTPUT

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

Insert your functions here.

In [1]:
# from A3mysolution import *
# Importing libraries
import numpy as np
import copy as copy
import math
import pandas


Depth Limited search algorithm computes the result based on other functions. 
* Initially, there are few checks to avoid integer, tuple errors.
* Then, there is an iteration to the number of valid actions. We try to find the best child state by recursively using the depthLimitedSearch function within the for loop, so it tests for every action.
* Then, we check is there is a cutoff or failure, if not we append the obtained child state in the result and return it.

Iterative Limited Search just calls the depth limited search function iteratively until the max depth is reached. We decrease the max depth within the depth limited search function


In [2]:
count_itr=0
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    
    global count_itr
    if type(state[0]) == int:
        state = printState_8p(state)
    if type(goalState[0]) == int:
        goalState = printState_8p(goalState)
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        count_itr+=1
        childState,_ = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result != 'failure':
            result.insert(0,childState)
            return result
    
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'       

def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    if type(startState[0]) == int:
        startState = printState_8p(startState)
    if type(goalState[0]) == int:
        goalState = printState_8p(goalState)
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':

            if type(startState) == str:
                result.insert(0,startState) 
            else:
                state = [item for sublist in startState for item in sublist]
                result.insert(0,state)   
            #print('result', result)
            return result
    return 'cutoff'


## 8-puzzle problem

Here, we have defined six functions.
* printState_8p - This function converts the original list in an array form so that we can get the exact position of the blank space.

* findBlank_8p - This function iterates through the array and find the blank space.

* actionState_8p - It is responsible for finding the valid actions based on the blank space. It's just made out of different checks.

* takeAction_8p - Based on the obtained valid action, it just swaps the position with the corresponding element and returns the list.

* goalTestF_8p - Returns true if goal == state

* findIndex - Used in heuristic functions to get the two dimensional index of the list.

In [3]:
def printState_8p(startState):
    state = [[startState[0],startState[1],startState[2]],[startState[3],startState[4],startState[5]],[startState[6],startState[7],startState[8]]]
    return state

def findBlank_8p(state):
    i = 0
    j = 0
    if type(state[0]) == int:
        state = printState_8p(state)
    for subi in state:
        for subj in subi:
            if state[i][j] == 0:
                return i,j
            j=j+1
        i=i+1
        j = 0
    i = 0

def findIndex_8p(state,index):
    k = 0
    for i in range(3):
        for j in range(3):
            if state[k]==index:
                return i,j
            k= k+1

def actionsF_8p(state):
    action_list = [('left',1), ('right',1), ('up',1),('down',1)]
    if type(state[0]) == int:
        state = printState_8p(state)    
    i,j = findBlank_8p(state)
    if i == 0 and j == 0:
        valid_action = [action_list[1], action_list[3]]
    if i == 2 and j == 2:
        valid_action = [action_list[0], action_list[2]]
    if i == 0 and j == 2:
        valid_action = [action_list[0], action_list[3]]
    if i == 2 and j == 0:
        valid_action = [action_list[1], action_list[2]]
    if  i == 1 and j == 0:
        valid_action = [action_list[1], action_list[2], action_list[3]]
    if i == 1 and j ==2:
        valid_action = [action_list[0], action_list[2], action_list[3]]
    if i == 0 and j == 1:
        valid_action = [action_list[0], action_list[1], action_list[3]]
    if i == 2 and j == 1:
        valid_action = [action_list[0], action_list[1], action_list[2]]
    if i == 1 and j == 1:
        valid_action = [action_list[0], action_list[1], action_list[2], action_list[3]]
    
    return valid_action

def takeActionF_8p(state2, action):
    if type(state2[0]) == int:
        state2 = printState_8p(state2)
    state1 = copy.deepcopy(state2)
    i,j = findBlank_8p(state1)
    if action == (('left',1)):
        temp = state1[i][j-1]
        state1[i][j-1] = state1[i][j]
        state1[i][j] = temp
        state1 = [item for sublist in state1 for item in sublist]
        return state1,1
    if action == (('right',1)):
        temp = state1[i][j+1]
        state1[i][j+1] = state1[i][j]
        state1[i][j] = temp
        state1 = [item for sublist in state1 for item in sublist]
        return state1,1
    if action == (('up',1)):
        temp = state1[i-1][j]
        state1[i-1][j] = state1[i][j]
        state1[i][j] = temp
        state1= [item for sublist in state1 for item in sublist]
        return state1, 1
    if action == (('down',1)):
        temp = state1[i+1][j]
        state1[i+1][j] = state1[i][j]
        state1[i][j] = temp
        state1 = [item for sublist in state1 for item in sublist]
        #print('state1', state1)
        return state1, 1

def goalTestF_8p(s, goal):
    return s == goal

def printPath_8p(startState, goalState, path):
    print(path)
    for p in path: 
        print ("\n",p[0],p[1],p[2],"\n",p[3],p[4],p[5],"\n",p[6],p[7],p[8])
        

        
    

## Heuristic Functions

For `aStarSearch`, I have defined four heuristics, first where h = 0, manhattan distance, rms and hamming distance  

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = sd$, where sd is the rms value obtained.
  * 'h4_8p(state,goal)' : $h(state, goal) = h$, where h is the hamming distance

In [4]:
def h1_8p(state, goal):                              # '0' heuristic function
    return 0 

def h2_8p(state, goal):                              # Manhattan distance
    m = 0
    for x in range (len(state)):
        i, j = findIndex_8p(state,x)
        s,t = findIndex_8p(goal,x)
        m = abs(s-i)+abs(t-j)
    return m

def h3_8p(state, goal):                             # root mean square
    sd = 0
    for x in range (len(state)):
        i, j = findIndex_8p(state,x)
        s,t = findIndex_8p(goal,x)
        sd = math.sqrt(math.pow((s-i),2)+math.pow((t-j),2))
    return sd

def h4_8p(state, goal):                             # hamming distance
    h = 0
    for i in range (len(state)):
        if state.index(i)!=goal.index(i):
            h = h+1
        return h

In the code below,
* Initialize all the necessary variables and make copies of nNodes so that we don't use updated values.
* Check if the number of nodes and depth is less than 0 and return 0 if it is so. This signifies that we dont have any branching.
* Now to iterate to find the optimum branching factor, we first start by taking the average of 0 and number of nodes thus limiting ebf between them.
* Next, we apply formula to find potential ebf.
* Save the values in dummy variables initialized before and make changes to reduce the ebf value or increase it based on the obtained b.
* return final value

In [5]:
def ebf(nNodes, depth, precision=0.01):
    minvalue=0
    maxvalue=nNodes
    maxrange=nNodes
    finalebf=1
    evalue=0
    final_ev=0
    #if depth > 0:
        #return 'failure'
    if nNodes<=0 or depth<0:
        return 0.0
  
    while minvalue<=maxvalue:
        b=(minvalue+maxvalue)/2
        if b!=1:
            evalue = (1-(b**(depth+1)))/(1-b)
        r=abs(evalue-nNodes)
        if r<maxrange:
            maxrange=r
            final_ev=evalue
            finalebf=b
        if r<precision:
            return finalebf
            
        if evalue<nNodes:
            minvalue=b
        if evalue>nNodes:
            maxvalue=b


## A* SEARCH ALGORITHM

In [6]:
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
#  Recursive Iterative Deepening form of A*, where depth is replaced by f(n)
count_astar = 0
class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):

    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global count_astar
    #count_astar = 0
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        count_astar+=1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":               #        g
            result.insert(0,parentNode.state)     #       / 
            return (result, bestChild.f)          #      d
                                                  #     / \ 
if __name__ == "__main__":                        #    b   h   
                                                  #   / \   
    successors = {'a': ['b','c'],                 #  a   e  
                  'b': ['d','e'],                 #   \         
                  'c': ['f'],                     #    c   i
                  'd': ['g', 'h'],                #     \ / 
                  'f': ['i','j']}                 #      f  
    def actionsF(s):                              #       \ 
        try:                                      #        j
            ## step cost of each action is 1
            return [(succ,1) for succ in successors[s]]
        except KeyError:
            return []
    def takeActionF(s,a):
        return a
    def goalTestF(s):
        return s == goal
    def h1(s):
        return 0
    start = 'a'
    goal = 'h'
    result = aStarSearch(start,actionsF,takeActionF,goalTestF,h1)
    print('Path from a to h is', result[0], 'for a cost of', result[1])

Path from a to h is ['a', 'b', 'd', 'h'] for a cost of 3


In this function, we are ust accumulating all the data by calling all the funtions designed above and print in a particular format

In [7]:
h1 = h1_8p
h2 = h2_8p
h3 = h3_8p
h4 = h4_8p
h = [h1, h2, h3, h4]
goalState1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]         
goalState2 = [1, 2, 3, 4, 5, 8, 6, 0, 7]
goalState3 = [1, 0, 3, 4, 5, 8, 2, 6, 7]

def runExperiment(goalState1, goalState2, goalState3, h):
    global count_itr
    global count_astar
    goals = [goalState1, goalState2, goalState3]
    for g in goals:
        count_itr = 0
        result_itr = iterativeDeepeningSearch([1, 2, 3, 4, 0, 5, 6, 7, 8], g, actionsF_8p, takeActionF_8p, 15)
        depth_itr = len(result_itr)-1
        ebf_itr = ebf(count_itr, depth_itr, 0.01)
        print (g)
        print('')
        print('Algorithm', ' ', 'Depth', ' ', 'Nodes', ' ', 'ebf')
        print ('IDS', '          ', depth_itr,'     ',count_itr,'  ', ebf_itr)
        for h_f in h:
            count_astar = 0
            result_astar = aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, g),
            lambda s: h_f(s, g))
            depth_astar = result_astar[1]
            ebf_astar = ebf(count_astar, depth_astar, 0.01)
            print (' ', 'astar_h' , '          ', depth_astar,'     ',count_astar,'  ', ebf_astar)
runExperiment(goalState1, goalState2, goalState3, h)   


[1, 2, 3, 4, 0, 5, 6, 7, 8]

Algorithm   Depth   Nodes   ebf
IDS            0       0    0.0
  astar_h            0       0    0.0
  astar_h            0       0    0.0
  astar_h            0       0    0.0
  astar_h            0       0    0.0
[1, 2, 3, 4, 5, 8, 6, 0, 7]

Algorithm   Depth   Nodes   ebf
IDS            3       43    3.0857696533203125
  astar_h            3       116    4.487663269042969
  astar_h            3       46    3.1680450439453125
  astar_h            3       46    3.1680450439453125
  astar_h            3       41    3.029205322265625
[1, 0, 3, 4, 5, 8, 2, 6, 7]

Algorithm   Depth   Nodes   ebf
IDS            11       225850    2.953883095619858
  astar_h            11       643246    3.262755963483812
  astar_h            11       190817    2.906762674106403
  astar_h            11       331528    3.063842615192243
  astar_h            11       368646    3.0949094382415865


## RESULTS

* We can observe that both iterative and A* gets the solution in the same number of steps.
* But we can see A* is much more extensive than iterative because of the number of nodes explored. But we can also observe that depending on the heuristic function the complexity of the algorithm changes. Thus, heuristic function becomes very important. Accuracy and optimization is solely dependent on this factor. EBF is the measure of these factors.
* Both seem to take similar amount of time on CPU. Probably a larger problem or higher precision would habe given us a better idea.

In [8]:
result

(['a', 'b', 'd', 'h'], 3)

In [9]:
ebf(count_astar, result[1],0.01)
count_astar

368646

The smallest argument values should be a depth of 0, and 1 node.

In [10]:
ebf(1, 0)

0.5

In [11]:
ebf(2, 1)

1.0078125

In [12]:
ebf(2, 1, precision=0.000001)

1.0000009536743164

In [13]:
ebf(200000, 5)

11.275596989435144

In [14]:
ebf(200000, 50)

1.234819248452368

Here is a simple example using our usual simple graph search.

In [15]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [16]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [17]:
takeActionF_simple('a', actions[0])

('b', 1)

In [18]:
goalTestF_simple('a', 'a')

True

In [19]:
h_simple('a', 'z')

1

In [20]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [21]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [22]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            